In [1]:
#@title Import Libraries {display-mode: "form"}
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import cv2
import math
import urllib
import warnings
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import add
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import constraints
from tensorflow.keras import backend as K
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras.preprocessing import image
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.applications.mobilenet import preprocess_input
from keras.applications.imagenet_utils import decode_predictions


Using TensorFlow backend.


In [0]:
#@title Configurations {display-mode: "form"}

BASE_WEIGHT_PATH = ('https://github.com/fchollet/deep-learning-models/'
                    'releases/download/v0.6/')

# Set log level for TensorFlow
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
#@title ReLU6 {display-mode: "form"}
def relu6(x):
    return K.relu(x, max_value=6)

In [0]:
#@title DepthwiseConv2D {display-mode: "form"}
class DepthwiseConv2D(Conv2D):
    def __init__(self,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 depth_multiplier=1,
                 data_format=None,
                 activation=None,
                 use_bias=True,
                 depthwise_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 depthwise_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 depthwise_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(DepthwiseConv2D, self).__init__(
            filters=None,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            activation=activation,
            use_bias=use_bias,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            bias_constraint=bias_constraint,
            **kwargs)
        self.depth_multiplier = depth_multiplier
        self.depthwise_initializer = initializers.get(depthwise_initializer)
        self.depthwise_regularizer = regularizers.get(depthwise_regularizer)
        self.depthwise_constraint = constraints.get(depthwise_constraint)
        self.bias_initializer = initializers.get(bias_initializer)

        self._padding = padding.upper()

        if K.image_data_format() == 'channels_last':
            self._strides = (1,) + strides + (1,)
        else:
            self._strides = (1, 1,) + strides

        if self.data_format == 'channels_last':
            self._data_format = "NHWC"
        else:
            self._data_format = "NCHW"

    def build(self, input_shape):
        if len(input_shape) < 4:
            raise ValueError('Inputs to `DepthwiseConv2D` should have rank 4. '
                             'Received input shape:', str(input_shape))
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = 3
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs to '
                             '`DepthwiseConv2D` '
                             'should be defined. Found `None`.')
        input_dim = int(input_shape[channel_axis])
        depthwise_kernel_shape = (self.kernel_size[0],
                                  self.kernel_size[1],
                                  input_dim,
                                  self.depth_multiplier)

        self.depthwise_kernel = self.add_weight(
            shape=depthwise_kernel_shape,
            initializer=self.depthwise_initializer,
            name='depthwise_kernel',
            regularizer=self.depthwise_regularizer,
            constraint=self.depthwise_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(input_dim * self.depth_multiplier,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        # Set input spec.
        self.input_spec = InputSpec(ndim=4, axes={channel_axis: input_dim})
        self.built = True

    def call(self, inputs, training=None):
        outputs = tf.nn.depthwise_conv2d(
            inputs,
            self.depthwise_kernel,
            strides=self._strides,
            padding=self._padding,
            data_format=self._data_format)

        if self.bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)

        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            rows = input_shape[2]
            cols = input_shape[3]
            out_filters = input_shape[1] * self.depth_multiplier
        elif self.data_format == 'channels_last':
            rows = input_shape[1]
            cols = input_shape[2]
            out_filters = input_shape[3] * self.depth_multiplier

        rows = conv_utils.conv_output_length(rows, self.kernel_size[0],
                                             self.padding,
                                             self.strides[0])
        cols = conv_utils.conv_output_length(cols, self.kernel_size[1],
                                             self.padding,
                                             self.strides[1])

        if self.data_format == 'channels_first':
            return (input_shape[0], out_filters, rows, cols)
        elif self.data_format == 'channels_last':
            return (input_shape[0], rows, cols, out_filters)

    def get_config(self):
        config = super(DepthwiseConv2D, self).get_config()
        config.pop('filters')
        config.pop('kernel_initializer')
        config.pop('kernel_regularizer')
        config.pop('kernel_constraint')
        config['depth_multiplier'] = self.depth_multiplier
        config['depthwise_initializer'] = initializers.serialize(self.depthwise_initializer)
        config['depthwise_regularizer'] = regularizers.serialize(self.depthwise_regularizer)
        config['depthwise_constraint'] = constraints.serialize(self.depthwise_constraint)
        
        return config

In [0]:
#@title Convolution Block and Depthwise Convolution Block {display-mode: "form"}
# taken from https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/conv_blocks.py
def _make_divisible(v, divisor=8, min_value=8):
    if min_value is None:
        min_value = divisor

    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def _conv_block(inputs, filters, alpha, kernel=(3, 3), strides=(1, 1), bn_epsilon=1e-3,
                bn_momentum=0.99, block_id=1):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    filters = filters * alpha
    filters = _make_divisible(filters)
    x = Conv2D(filters, kernel,
               padding='same',
               use_bias=False,
               strides=strides,
               name='conv%d' % block_id)(inputs)
    x = BatchNormalization(axis=channel_axis, momentum=bn_momentum, epsilon=bn_epsilon,
                           name='conv%d_bn' % block_id)(x)
    
    return Activation(relu6, name='conv%d_relu' % block_id)(x)


def _depthwise_conv_block(inputs, pointwise_conv_filters, alpha,
                          depth_multiplier=1, strides=(1, 1),
                          bn_epsilon=1e-3, block_id=1):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    pointwise_conv_filters = _make_divisible(pointwise_conv_filters * alpha)

    x = DepthwiseConv2D((3, 3),
                        padding='same',
                        depth_multiplier=depth_multiplier,
                        strides=strides,
                        use_bias=False,
                        name='conv_dw_%d' % block_id)(inputs)
    x = BatchNormalization(axis=channel_axis, epsilon=bn_epsilon, name='conv_dw_%d_bn' % block_id)(x)
    x = Activation(relu6, name='conv_dw_%d_relu' % block_id)(x)

    x = Conv2D(pointwise_conv_filters, (1, 1),
               padding='same',
               use_bias=False,
               strides=(1, 1),
               name='conv_pw_%d' % block_id)(x)
    x = BatchNormalization(axis=channel_axis, epsilon=bn_epsilon, name='conv_pw_%d_bn' % block_id)(x)
    
    return Activation(relu6, name='conv_pw_%d_relu' % block_id)(x)

In [0]:
input_your_code = None


'''MobileNet from keras.applications
'''
def MobileNet(input_shape=None,
              alpha=1.0,
              depth_multiplier=1,
              dropout=1e-3,
              include_top=True,
              weights='imagenet',
              input_tensor=None,
              pooling=None,
              classes=1000):

    # MobileNet requires 224x224x3 input size.
    if input_shape is None:
        default_size = 224

    '''Expected input_shape is (224, 224, 3).
    Here we use _obtain_input_shape from keras.applications.imagenet_utils
    to determine input_shape.
    '''
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=default_size,
                                      min_size=32,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top or weights)

    # Check if channels locate at the last dimension.
    if K.image_data_format() == 'channels_last':
        row_axis, col_axis = (0, 1)
    else:
        row_axis, col_axis = (1, 2)
    # Number of rows: 224
    rows = input_shape[row_axis]
    # Number of columns: 224
    cols = input_shape[col_axis]

    if input_tensor is None:
        '''Input layer from keras.layers:
        https://www.tensorflow.org/api_docs/python/tf/keras/layers/Input
        input_shape: (224, 224, 3)
        Input() gets an input as NumPy array and returns a tensor.
        '''
        img_input = Input(shape=input_shape)
    else:
        # In case you input a tensor into Input layer.
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    '''The first convolutional block with kernel size 3x3 by default.
    The number of filters are 32.
    
    Alpha controls the width of the network. If alpha is 1,
    default number of filters from the paper are used at each layer.
    
    depth_multiplier controls number of output channels of depthwise convolution.
    The default depth_multiplier is 1. The output channels is depth_multiplier
    times number of channels (filters) from the previous layer.
    
    Stride 2 reduces the size of the output by 2,
    it reduces runtime for training and testing.
    '''
    # The first convolution block
    # Get the input from img_input
    # _conv_block, number of filters 32, alpha 1, stride 2
    x = input_your_code

    # alpha and depth_multiplier are always 1
    # Depthwise convolution block_id 1
    # _depthwise_conv_block
    # number of filters 64, stride 1
    x = input_your_code
    
    # Depthwise convolution block_id 2
    # _depthwise_conv_block
    # number of filters 128, stride 2
    x = input_your_code
    
    # Depthwise convolution block_id 3
    # _depthwise_conv_block
    # number of filters 128, stride 1
    x = input_your_code

    # Depthwise convolution block_id 4
    # _depthwise_conv_block
    # number of filters 256, stride 2
    x = input_your_code
    
    
    # Depthwise convolution block_id 5
    # _depthwise_conv_block
    # number of filters 256, stride 1
    x = input_your_code

    # Depthwise convolution block_id 6
    # _depthwise_conv_block
    # number of filters 512, stride 2
    x = input_your_code
    
    # Depthwise convolution block_id 7, 8, 9, 10, and 11
    # _depthwise_conv_block
    # number of filters 512, stride 1
    x = input_your_code
    x = input_your_code
    x = input_your_code
    x = input_your_code
    x = input_your_code

    # Depthwise convolution block_id 12
    # _depthwise_conv_block
    # number of filters 1024, stride 2
    x = input_your_code
    
    # Depthwise convolution block_id 13
    # _depthwise_conv_block
    # number of filters 1024, stride 1
    x = input_your_code

    # Classification layers
    if include_top:
        # shape for flatten (Reshape) into 1x1x1024.
        shape = (1, 1, int(1024 * alpha))

        x = GlobalAveragePooling2D()(x)
        x = Reshape(shape, name='reshape_1')(x)
        x = Dropout(dropout, name='dropout')(x)
        x = Conv2D(classes, (1, 1),
                   padding='same', name='conv_preds')(x)
        x = Activation('softmax', name='act_softmax')(x)
        # classes = 1000
        x = Reshape((classes,), name='reshape_2')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create a model.
    model = Model(inputs, x, name='mobilenet_%0.2f_%s' % (alpha, rows))

    # Load weights from ImageNet.
    if weights == 'imagenet':
        if K.image_data_format() == 'channels_first':
            raise ValueError('Weights for "channels_last" format '
                             'are not available.')
        if alpha == 1.0:
            alpha_text = '1_0'
        elif alpha == 0.75:
            alpha_text = '7_5'
        elif alpha == 0.50:
            alpha_text = '5_0'
        else:
            alpha_text = '2_5'

        if include_top:
            model_name = 'mobilenet_%s_%d_tf.h5' % (alpha_text, rows)
            weigh_path = BASE_WEIGHT_PATH + model_name
            weights_path = get_file(model_name,
                                    weigh_path,
                                    cache_subdir='models')
        else:
            model_name = 'mobilenet_%s_%d_tf_no_top.h5' % (alpha_text, rows)
            weigh_path = BASE_WEIGHT_PATH + model_name
            weights_path = get_file(model_name,
                                    weigh_path,
                                    cache_subdir='models')
        model.load_weights(weights_path)

    return model

In [7]:
# my_model = MobileNet(alpha=1, depth_multiplier=1)
'''There are mean and variance for non-trainable parameters.
They are updated separately from backpropagation.
You can check those mean and variance from my_model.non_trainable_weights.
'''
# my_model.summary()

'There are mean and variance for non-trainable parameters.\nThey are updated separately from backpropagation.\nYou can check those mean and variance from my_model.non_trainable_weights.\n'

In [0]:
!mkdir -p /usr/local/share/jupyter/nbextensions/google.colab/images
!rm -f /usr/local/share/jupyter/nbextensions/google.colab/images/*.jpg

In [0]:
def url_to_image(url):
  # download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  I = image
  if (len(I.shape))==3: #check if the image has width, length and channels, as I found some withouth channel
    save_path = '/usr/local/share/jupyter/nbextensions/google.colab/images/image.jpg'
    cv2.imwrite(save_path,I)
	
  return image

In [0]:
# Change image URL here:
image_url = 'https://i.pinimg.com/236x/b8/02/36/b80236633d69c017ea04b08a1f01e665--hair-chalk-hair-dye.jpg'

In [0]:
# A list of ImageNet classes: http://image-net.org/challenges/LSVRC/2012/browse-synsets
# enter the url of the .jpg image
actual_image = url_to_image(image_url)
img_path = '/usr/local/share/jupyter/nbextensions/google.colab/images/image.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.

In [12]:
%%html
<h2>An input image:</h2><br>
<img src='/nbextensions/google.colab/images/image.jpg' />

In [0]:
# preds = my_model.predict(x)

# print('Prediction Results\n')
# for prediction in decode_predictions(preds)[0]:
#   print('{}: {:.2f}%'.format(
#       prediction[1],
#       prediction[2] * 100)
#   )